<a href="https://colab.research.google.com/github/aminik/projects_GoogleColab/blob/main/model_nn_transaction_netron_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 9. Использование CPU и GPU на примере простой НС - Нейросети на PyTorch

In [ ]:
!ls sample_data/*.csv



sample_data/california_housing_test.csv   sample_data/mnist_test.csv
sample_data/california_housing_train.csv  sample_data/mnist_train_small.csv


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
data = load_iris()
X = data['data']
y = data['target']
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)# split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

In [3]:
class IrisModel(nn.Module):
	def __init__(self):
		super().__init__()
		self.hidden = nn.Linear(4, 8)
		self.act = nn.ReLU()
		self.output = nn.Linear(8, 3)
	def forward(self, x):
		x = self.act(self.hidden(x))
		x = self.output(x)
		return x

In [4]:
# loss metric and optimizer
model = IrisModel()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# prepare model and training parameters
n_epochs = 100
batch_size = 10
batch_start = torch.arange(0, len(X_train), batch_size)

In [5]:
# training loop
for epoch in range(n_epochs):
	for start in batch_start:        # take a batch
		X_batch = X_train[start:start+batch_size]
		y_batch = y_train[start:start+batch_size]
		# forward pass
		y_pred = model(X_batch)
		loss = loss_fn(y_pred, y_batch)
		# backward pass
		optimizer.zero_grad()
		loss.backward()
		# update weights
		optimizer.step()

In [6]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.6 MB/s eta 0:00:00


In [7]:
# validating 		model
y_pred = model(X_test)
acc = (torch.argmax(y_pred, 1) == y_test).float().mean()
acc = float(acc)*100
print("Model accuracy: %.2f%%" % acc)

Model accuracy: 100.00%


In [9]:
torch.onnx.export(model, X_test, 'iris.onnx', input_names=["features"], output_names=["logits"])

In [ ]:
Выполнение этой функции создаст файл iris.onnx в локальном каталоге. Вам необходимо предоставить пример тензора, который работает с моделью в качестве входных данных (X_test в приведенном выше примере).

Это необходимо потому, что во время преобразования необходимо следовать этому примеру тензора, чтобы понять, какие операции должны быть применены, чтобы вы могли преобразовать алгоритм шаг за шагом в формат ONNX.

Каждый вес в модели PyTorch является тензором, и им присвоено имя. Но входные и выходные тензоры обычно не имеют имен, поэтому вам необходимо задать им имя при выполнении функции export(). Эти имена быть представлены в виде списка строк, поскольку в общем случае модель может принимать несколько тензоров и возвращать несколько тензоров.

Обычно функцию export() следует запускать после цикла обучения. Это связано с тем, что созданная модель ONNX содержит полную модель, которую можно запустить без библиотеки PyTorch. Вы хотите сохранить оптимизированный вес в ней. Однако для целей визуализации модели в Netron качество модели не имеет значения. Вы можете вызвать функцию export(), как только модель PyTorch будет создана.

In [10]:
!ls -ah


.  ..  .config	iris.onnx  sample_data


In [12]:
!pip install netron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
